In [1]:
from pydantic_ai import Agent, Tool
from pydantic_ai.exceptions import ModelRetry
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="env/.env")

# Create the model with OpenRouter configuration
api_key = os.environ.get("OPENROUTER_API_KEY")
base_url = os.environ.get("OPENROUTER_BASE_URL")

model = OpenAIModel(
    model_name="gpt-3.5-turbo",
    provider=OpenAIProvider(base_url=base_url, api_key=api_key),
)

In [ ]:
async def plan_holiday(destination: str, duration: str):
    """Plan a holiday."""
    return f"Holiday plan completed: {destination} for {duration} days. The plan is ready for booking."

my_tool = Tool(
    name="holiday_planner",
    function=plan_holiday,
    description="Plan a complete holiday with destination and duration. Return the final plan ready for booking."
)

# Create agent
agent = Agent(
    model=model,
    tools=[my_tool],
    instructions="You are a helpful holiday planning assistant. Use tools when needed."
)

In [3]:
async def request_holiday_confirmation(context):
    tool_call = context['tool_call']
    
    # Get the holiday details from the tool call
    args = tool_call.args_as_dict()
    destination = args.get('destination', 'unknown')
    duration = args.get('duration', 'unknown')
    
    print(f"Please confirm with destionation: {destination} and duration: {duration}")
    print("┌─────────────┬────────────┐")
    print("│    [y]es    │    [n]o    │")
    print("└─────────────┴────────────┘")
    
    user_input = input("Enter y or n: ").lower().strip()
    
    if user_input == 'y':
        print("✅ Holiday plan approved - user pressed [y]")
        return None  # Allow the tool to execute
    elif user_input == 'n':
        print("❌ Holiday plan denied - user pressed [n]")
        return ModelRetry("Holiday plan was denied by user. Ask help from the user.")
    

async def log_after(context):
    tool_call = context['tool_call']
    print(f"🔒 {tool_call.tool_name} approved {tool_call.args}")

agent.on.tool(my_tool).before = request_holiday_confirmation
agent.on.tool(my_tool).after = log_after

In [4]:
# Test the holiday planner
await agent.run("Plan a holiday to Switzerland for 7 days")

Please confirm with destionation: Switzerland and duration: 7 days
┌─────────────┬────────────┐
│    [y]es    │    [n]o    │
└─────────────┴────────────┘
✅ Holiday plan approved - user pressed [y]
🔒 holiday_planner approved {"destination":"Switzerland","duration":"7 days"}


AgentRunResult(output='I have planned a holiday to Switzerland for 7 days. The plan is ready for booking.')

In [5]:
# Test the holiday planner
await agent.run("Plan a holiday to Switzerland for 7 days")

Please confirm with destionation: Switzerland and duration: 7 days
┌─────────────┬────────────┐
│    [y]es    │    [n]o    │
└─────────────┴────────────┘
❌ Holiday plan denied - user pressed [n]


AgentRunResult(output="I'm sorry that the holiday plan was not approved. Let me know how I can assist you better with planning your holiday to Switzerland for 7 days.")